In [15]:
import re 
from lightrag.core import Component, Generator, DataClass
from lightrag.components.model_client import OpenAIClient
from lightrag.components.model_client import GroqAPIClient

# a router and two generators
# every single component might need a signature.
# TODO: LLM for single choices
class MyChatBot(Component):
        def __init__(self):
            super().__init__()
            model_1_kwargs = {
                "model": "gpt-3.5-turbo",
            }
            model_2_kwargs = {
                 "model": "llama3-8b-8192"
            }
            template_doc = r"""<SYS> You are a doctor </SYS> User: {{input_str}}"""
            template_law = r"""<SYS> You are a lawyer </SYS> User: {{input_str}}"""
            self.doc = Generator(template=template_doc, model_client=OpenAIClient(), model_kwargs=model_1_kwargs)
            self.lawyer = Generator(template=template_law, model_client=GroqAPIClient(), model_kwargs=model_2_kwargs)
            self.router_choices = {
                "doctor": self.create_generator_signature(self.doc),
                "lawyer": self.create_generator_signature(self.lawyer),
                "other": "Choose me the question does not apply to other choices."
            }
            print(self.router_choices)
            template_router = r"""<SYS> You are a router who will route a user question to the right generator.
            Here are your choices in form of key: value pairs:
             {% for key, value in choices.items() %}
                {{ key }}: {{ value }}
             {% endfor %}
            Output the key of your choice.
            </SYS> User question: {{input_str}}
            You:
            """
            self.router = Generator(template=template_router, model_client=OpenAIClient(), model_kwargs=model_1_kwargs)

            # self.router = 
        def create_generator_signature(self, generator: Generator):
            template = generator.template
            pattern = r"<SYS>(.*?)</SYS>"

            matches = re.findall(pattern, template)
            # Print the contents found between <SYS> tags
            for match in matches:
                print("Content between <SYS> tags:", match)
                return match
        
        
        def call(self, query: str):
            choice = self.router(prompt_kwargs={"input_str": query, "choices": self.router_choices}).data
            print(f"Choice: {choice}")
            if choice == "doctor":
                return self.doc(prompt_kwargs={"input_str": query}).data
            elif choice == "lawyer":
                return self.lawyer(prompt_kwargs={"input_str": query}).data
            else:
                return "Choose me the question does not apply to other choices."

Initiate the task component

In [16]:
task = MyChatBot()


Content between <SYS> tags:  You are a doctor 
Content between <SYS> tags:  You are a lawyer 
{'doctor': ' You are a doctor ', 'lawyer': ' You are a lawyer ', 'other': 'Choose me the question does not apply to other choices.'}


Visualize the structure of generator component and task component

In [8]:
task.doc

Generator(
  model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
  (system_prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
  (model_client): OpenAIClient()
)

In [3]:
task.lawyer

Generator(
  model_kwargs={'model': 'llama3-8b-8192'}, model_type=ModelType.LLM
  (system_prompt): Prompt(template: <SYS> You are a lawyer </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
  (model_client): GroqAPIClient()
)

In [9]:
task.router

Generator(
  model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
  (system_prompt): Prompt(
    template: <SYS> You are a router who will route a user question to the right generator.
                Here are your choices in form of key: value pairs:
                 {% for key, value in choices.items() %}
                    {{ key }}: {{ value }}
                 {% endfor %}
                Output the key of your choice.
                </SYS> User question: {{input_str}}
                You:
                , prompt_variables: ['input_str', 'choices']
  )
  (model_client): OpenAIClient()
)

In [10]:
task

MyChatBot(
  (doc): Generator(
    model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
    (system_prompt): Prompt(template: <SYS> You are a doctor </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
    (model_client): OpenAIClient()
  )
  (lawyer): Generator(
    model_kwargs={'model': 'llama3-8b-8192'}, model_type=ModelType.LLM
    (system_prompt): Prompt(template: <SYS> You are a lawyer </SYS> User: {{input_str}}, prompt_variables: ['input_str'])
    (model_client): GroqAPIClient()
  )
  (router): Generator(
    model_kwargs={'model': 'gpt-3.5-turbo'}, model_type=ModelType.LLM
    (system_prompt): Prompt(
      template: <SYS> You are a router who will route a user question to the right generator.
                  Here are your choices in form of key: value pairs:
                   {% for key, value in choices.items() %}
                      {{ key }}: {{ value }}
                   {% endfor %}
                  Output the key of your choice.
            

In [18]:
from lightrag.utils import enable_library_logging
enable_library_logging()

In [19]:
# call 

task.call("I have a legal question")

2024-06-09 11:38:42 - INFO - [generator.py:194:call] - prompt_kwargs: {'input_str': 'I have a legal question', 'choices': {'doctor': ' You are a doctor ', 'lawyer': ' You are a lawyer ', 'other': 'Choose me the question does not apply to other choices.'}}
2024-06-09 11:38:42 - INFO - [generator.py:195:call] - model_kwargs: {}
2024-06-09 11:38:42 - INFO - [openai_client.py:122:call] - api_kwargs: {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': '<SYS> You are a router who will route a user question to the right generator.\n            Here are your choices in form of key: value pairs:\n                doctor:  You are a doctor \n                lawyer:  You are a lawyer \n                other: Choose me the question does not apply to other choices.\n            Output the key of your choice.\n            </SYS> User question: I have a legal question\n            You:'}]}
2024-06-09 11:38:42 - INFO - [_client.py:1026:_send_single_request] - HTTP Request: POST https:

'Choose me the question does not apply to other choices.'